In [40]:
using LinearAlgebra

In [3]:
mat1 = Material(k, rho, Cp)
mat2 = Material(...)

model = Model()

add_wall(model
         label="mur ext",    # not an id
         thickness=.2, #  m
         surface=3,    #  m2
         material=mat1)

add_wall(model
         name="mur int",   
         thickness=.3,
         material=mat1)
--> create node  mux_ext, mur_int

add_simple_mass(model, name, mass)

connect(nodeA, nodeB) -->

or
stack( wallA, wallB)

UndefVarError: UndefVarError: Material not defined

In [ ]:
add_conduction_surfacique(R, surface, nodeA, nodeB)
add_conduction_line(phi, length, nodeA, nodeB)

add_heat_source(model, node, phi(t))

fix_temperature(nodeC, T(t))

In [ ]:
add_convection(h,
               surface, 
               "nodeA",
                get_T_ext)

In [1]:
struct SimpleThermalMass
   mass_th # rhoCp V 
   label
end

struct Wall
   mat
   thickness
   label
end

In [3]:
SimpleThermalMass(mass_th) = SimpleThermalMass(mass_th, "")
Wall(mat, thickness) = Wall(mat, thickness, "")

Wall

In [7]:
struct Model
    simple_masses
    walls
    sources
    resistances
end

Model() = Model(Dict(), Dict(), Dict(), [])

Model

Model(Dict{Any,Any}(), Dict{Any,Any}(), Dict{Any,Any}(), Any[])

In [33]:
function add!(m::Model, simplemass::SimpleThermalMass; id=nothing)
    if id == nothing
        id = "node$(length(m.simple_masses)+1)"
        println("create node $id")
    end
    m.simple_masses[id] = simplemass
    end

add! (generic function with 2 methods)

In [34]:
m = Model()

Model(Dict{Any,Any}(), Dict{Any,Any}(), Dict{Any,Any}(), Any[])

In [38]:
add!(m, SimpleThermalMass(3.3); id="air_int")
add!(m, SimpleThermalMass(3.4))

create node node2


SimpleThermalMass(3.4, "")

In [39]:
Convection(Sh, nodeA, nodeB)
SourceConvection(Sh, node, T_ext(t, deltaT))

Model(Dict{Any,Any}("node2"=>SimpleThermalMass(3.4, ""),"air_int"=>SimpleThermalMass(3.3, "")), Dict{Any,Any}(), Dict{Any,Any}(), Any[])

In [43]:
function build(wall::Wall, dt)
    """-- Build the adiabatic wall matrix --"""
    delta_x = sqrt( dt * wall.mat.alpha )
    N = 1 + Int(ceil( wall.thickness/delta_x ))
    dx = thickness / N
    
    M = Tridiagonal(ones(N-1), -2.0 * ones(N), ones(N-1));
    #M[1, 1] += -2.0*h_left*dx/mat.k
    M[1, 2] = 2.0
    #M[end, end] += -2.0*h_right*dx/mat.k
    M[end, end-1] = 2.0

    A = mat.alpha / dx^2 * M;
    return M
end

build (generic function with 1 method)

In [ ]:
w = Wall(mat, thickness=0.2, label="ext")
add!(model, w)

SourceConvection(Sh, w.left, t->T_ext(t, deltaT))

In [ ]:
function assemble(m::Model, dt)
    
    M_walls = [build(w, dt) for w in m.walls]
    
    # convection
    M[ , ] += -2.0*conv.hS/m.conv.node